# 自动微分

[![在线运行](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/resource/_static/logo_modelarts.png)](https://authoring-modelarts-cnnorth4.huaweicloud.com/console/lab?share-url-b64=aHR0cHM6Ly9vYnMuZHVhbHN0YWNrLmNuLW5vcnRoLTQubXlodWF3ZWljbG91ZC5jb20vbWluZHNwb3JlLXdlYnNpdGUvbm90ZWJvb2svcjEuNy90dXRvcmlhbHMvemhfY24vYmVnaW5uZXIvbWluZHNwb3JlX2F1dG9ncmFkLmlweW5i&imageid=9d63f4d1-dc09-4873-b669-3483cea777c0)&emsp;[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/r1.7/tutorials/zh_cn/beginner/mindspore_autograd.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/r1.7/tutorials/zh_cn/beginner/mindspore_autograd.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r1.7/tutorials/source_zh_cn/beginner/autograd.ipynb)

自动微分能够计算可导函数在某点处的导数值，是反向传播算法的一般化。自动微分主要解决的问题是将一个复杂的数学运算分解为一系列简单的基本运算，该功能对用户屏蔽了大量的求导细节和过程，大大降低了框架的使用门槛。

MindSpore使用`ops.GradOperation`计算一阶导数，`ops.GradOperation`属性如下：

+ `get_all`：计算梯度，如果等于False，获得第一个输入的梯度，如果等于True，获得所有输入的梯度。默认值为False。
+ `get_by_list`：是否对权重参数进行求导，默认值为False。
+ `sens_param`：是否对网络的输出值做缩放以改变最终梯度，默认值为False。

本章使用MindSpore中的`ops.GradOperation`对函数 $f(x)=wx+b$ 求一阶导数。

## 对输入求一阶导

对输入求导前需要先定义公式：

$$f(x)=wx+b \tag {1} $$

下面示例代码是公式(1)的表达，由于MindSpore采用函数式编程，因此所有计算公式表达都采用函数进行表示。

In [3]:
import numpy as np
import mindspore.nn as nn
from mindspore import Parameter, Tensor

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = Parameter(np.array([6.0]), name='w')
        self.b = Parameter(np.array([1.0]), name='b')

    def construct(self, x):
        f = self.w * x + self.b
        return f

然后定义求导类`GradNet`，类的`__init__`函数中定义需要求导的网络`self.net`和`ops.GradOperation`操作，类的`construct`函数中对`self.net`的输入进行求导。其对应MindSpore内部会产生如下公式(2)：

$$f^{'}(x)=w\tag {2}$$

In [4]:
from mindspore import dtype as mstype
import mindspore.ops as ops

class GradNet(nn.Cell):
    def __init__(self, net):
        super(GradNet, self).__init__()
        self.net = net
        self.grad_op = ops.GradOperation()

    def construct(self, x):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x)

最后定义权重参数为w，并对输入公式(1)中的输入参数x求一阶导数。从运行结果来看，公式(1)中的输入为6，即：

$$f(x)=wx+b=6*x+1 \tag {3}$$

对上式进行求导，有：

$$f^{'}(x)=w=6 \tag {4}$$

In [5]:
x = Tensor([100,200],dtype=mstype.float32)
output = GradNet(Net())(x)

print(output)

[6. 6.]


MindSpore计算一阶导数方法`ops.GradOperation(get_all=False, get_by_list=False, sens_param=False)`，其中`get_all`为`False`时，只会对第一个输入求导，为`True`时，会对所有输入求导。

## 对权重求一阶导

对权重参数求一阶导，需要将`ops.GradOperation`中的`get_by_list`设置为`True`。

In [6]:
from mindspore import ParameterTuple

class GradNet(nn.Cell):
    def __init__(self, net):
        super(GradNet, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)  # 设置对权重参数进行一阶求导

    def construct(self, x):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x)

接下来，对权重参数w进行求导：

f(w) = wx+b

f'(w) = x

In [7]:
# 对函数进行求导计算
x = Tensor([5], dtype=mstype.float32)
fx = GradNet(Net())(x)

# 打印结果
#print(f"wgrad: {fx[0]}\nbgrad: {fx[1]}")
print(fx)
print(fx[0])

(Tensor(shape=[1], dtype=Float64, value= [5.00000000e+000]), Tensor(shape=[1], dtype=Float64, value= [1.00000000e+000]))
[5.]


若某些权重不需要进行求导，则在定义求导网络时，相应的权重参数声明定义的时候，将其属性`requires_grad`需设置为`False`。

In [9]:
class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = Parameter(Tensor(np.array([6], np.float32)), name='w')
        self.b = Parameter(Tensor(np.array([1.0], np.float32)), name='b', requires_grad=False)

    def construct(self, x):
        out = x * self.w + self.b
        return out

class GradNet(nn.Cell):
    def __init__(self, net):
        super(GradNet, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)

    def construct(self, x):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x)

# 构建求导网络
x = Tensor([5], dtype=mstype.float32)
fw = GradNet(Net())(x)

print(fw)

(Tensor(shape=[1], dtype=Float32, value= [5.00000000e+000]),)


## 梯度值缩放

通过`sens_param`参数对网络的输出值做缩放以改变最终梯度。首先将`ops.GradOperation`中的`sens_param`设置为`True`，并确定缩放指数，其维度与输出维度保持一致。

In [8]:
class GradNet(nn.Cell):
    def __init__(self, net):
        super(GradNet, self).__init__()
        self.net = net
        # 求导操作
        self.grad_op = ops.GradOperation(sens_param=True)
        # 缩放指数
        self.grad_wrt_output = Tensor([2], dtype=mstype.float32)

    def construct(self, x):
        gradient_function = self.grad_op(self.net)
        return gradient_function(x, self.grad_wrt_output)

x = Tensor([6], dtype=mstype.float32)
output = GradNet(Net())(x)

print(output)

TypeError: Type Join Failed: dtype1 = Float32, dtype2 = Float64.
For more details, please refer to https://www.mindspore.cn/search?inputValue=Type%20Join%20Failed

Inner Message:
This: AbstractScalar(Type: Float32, Value: AnyValue, Shape: NoShape), other: AbstractScalar(Type: Float64, Value: AnyValue, Shape: NoShape). Please check the node: @Net.28:29{[0]: 29, [1]: Net}
The function call stack:
In file C:\Users\15405\AppData\Local\Temp\ipykernel_19068\3168270581.py:12/

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore\core\abstract\abstract_value.cc:65 TypeJoinLogging


## 停止计算梯度

使用`ops.stop_gradient`可以停止计算梯度，示例如下：

In [21]:
from mindspore.ops import stop_gradient

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.w = Parameter(Tensor(np.array([6], np.float32)), name='w')
        self.b = Parameter(Tensor(np.array([1.0], np.float32)), name='b')

    def construct(self, x):
        out = x * self.w + self.b
        # 停止梯度更新，out对梯度计算无贡献
        out = stop_gradient(out)
        return out

class GradNet(nn.Cell):
    def __init__(self, net):
        super(GradNet, self).__init__()
        self.net = net
        self.params = ParameterTuple(net.trainable_params())
        self.grad_op = ops.GradOperation(get_by_list=True)

    def construct(self, x):
        gradient_function = self.grad_op(self.net, self.params)
        return gradient_function(x)

x = Tensor([100], dtype=mstype.float32)
output = GradNet(Net())(x)

print(f"wgrad: {output[0]}\nbgrad: {output[1]}")

wgrad: [0.]
bgrad: [0.]
